In [1]:
#%matplotlib ipympl
%load_ext autoreload
%autoreload 2
import grimpfunc as grimp
import nisardev as nisar
import panel as pn
pn.extension()
#from matplotlib import colors
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import os
import matplotlib.pyplot as plt
import numpy as np
import dask
from dask.diagnostics import ProgressBar
ProgressBar().register()
dask.config.set(num_workers=2)  # Avoid too many NSIDC connections
import matplotlib.gridspec as gridspec

In [4]:
# Set path for gdal 
env = dict(GDAL_HTTP_COOKIEFILE = os.path.expanduser('~/.grimp_download_cookiejar.txt'),
            GDAL_HTTP_COOKIEJAR = os.path.expanduser('~/.grimp_download_cookiejar.txt'))
os.environ.update(env)
# Get login
myLogin = grimp.NASALogin()  # If login appears not to work, try rerunning this cell
myLogin.username = #type nasa earthdata username
myLogin.password = '#type nasa earthdata password
myLogin.view()

Getting login from ~/.netrc
Already logged in. Proceed.


In [5]:
# This is for Fjord 36
bbox = {'minx': 168500, 'miny': -2902476, 'maxx': 232012, 'maxy': -2871000}  # Modifiy values as needed
xbox = np.array([bbox[x] for x in ['minx', 'minx', 'maxx', 'maxx', 'minx']]) * 0.001
ybox = np.array([bbox[y] for y in ['miny', 'maxy', 'maxy', 'miny', 'miny']]) * 0.001

In [46]:
myImageUrls = grimp.cmrUrls(mode='image')  # mode image restricts search to the image products
myImageUrls.initialSearch(firstDate='2015-01-01', lastDate='2015-02-28', productFilter='image') # change the time window as needed
# selecting productFilter='sigma0' or 'gamma0' gives you Sigma0 or Gamma0 images which might be better for digital processing
# "image" is a "visualization" product

Row
    [0] Column(min_width=300)
        [0] Markdown(str)
        [1] Param(cmrUrls, name='Select Product &..., widgets={'product': <class 'panel....}, width=300)
        [2] Row
            [0] Markdown(str)
            [1] Markdown(str)
    [1] Column
        [0] ParamMethod(method, _pane=DataFrame)
        [1] ParamMethod(method, _pane=Markdown)
        [2] ParamMethod(method, _pane=Markdown)

In [47]:
myImageSeries = nisar.nisarImageSeries()  # Instantiate the series object
myImageSeries.readSeriesFromTiff(myImageUrls.getCogs(), chunkSize=2048)  # In this case increasing the chunksize can improve performance
myImageSeries.subset   # Display map of data layout - add ; to suppress this output

<xarray.DataArray 'image' (time: 5, band: 1, y: 106440, x: 59040)>
dask.array<concatenate, shape=(5, 1, 106440, 59040), dtype=uint8, chunksize=(1, 1, 2048, 2048), chunktype=numpy.ndarray>
Coordinates:
  * x            (x) float64 -6.26e+05 -6.26e+05 -6.259e+05 ... 8.5e+05 8.5e+05
  * y            (y) float64 -6.95e+05 -6.95e+05 ... -3.356e+06 -3.356e+06
  * time         (time) datetime64[ns] 2015-01-06T12:00:00 ... 2015-02-23T12:...
  * band         (band) <U5 'image'
    name         <U4 'None'
    _FillValue   int64 0
    time1        (time) datetime64[ns] 2015-01-01 2015-01-13 ... 2015-02-18
    time2        (time) datetime64[ns] 2015-01-12 2015-01-24 ... 2015-03-01
    spatial_ref  int64 0

In [48]:
myImageSeries.subset   # Display map of data layout - add ; to suppress this output

<xarray.DataArray 'image' (time: 5, band: 1, y: 106440, x: 59040)>
dask.array<concatenate, shape=(5, 1, 106440, 59040), dtype=uint8, chunksize=(1, 1, 2048, 2048), chunktype=numpy.ndarray>
Coordinates:
  * x            (x) float64 -6.26e+05 -6.26e+05 -6.259e+05 ... 8.5e+05 8.5e+05
  * y            (y) float64 -6.95e+05 -6.95e+05 ... -3.356e+06 -3.356e+06
  * time         (time) datetime64[ns] 2015-01-06T12:00:00 ... 2015-02-23T12:...
  * band         (band) <U5 'image'
    name         <U4 'None'
    _FillValue   int64 0
    time1        (time) datetime64[ns] 2015-01-01 2015-01-13 ... 2015-02-18
    time2        (time) datetime64[ns] 2015-01-12 2015-01-24 ... 2015-03-01
    spatial_ref  int64 0

In [49]:
myImageSeries.subSetImage(bbox)
myImageSeries.subset
myImageSeries.loadRemote()

[########################################] | 100% Completed | 3.05 sms


In [56]:
myImageSeries.subset[2,0,:,:]

TypeError: RasterArray.clip() missing 1 required positional argument: 'geometries'

In [51]:
#mySingleImageSeries = myImageSeries.timeSliceImage('2019-01-12', '2019-01-12')

In [52]:
# save it to netcdf file

In [54]:
myImageSeries.toNetCDF('Fjord36.subset.nc')

'Fjord36.subset.nc'